In [1]:
import os

DATA_FOLDER = os.path.abspath(os.path.join('..', 'data'))
FIGURE_FOLDER = os.path.abspath(os.path.join('..', 'figures'))

notebook_name = '021_get_nucleus_cytoplasm_sequences'

data_folder = os.path.join(DATA_FOLDER, notebook_name)
figure_folder = os.path.join(FIGURE_FOLDER, notebook_name)

! mkdir -p $data_folder
! mkdir -p $figure_folder

input_folder = os.path.join(DATA_FOLDER, '020_get_nucleus_cytoplasm_genes')

In [2]:
import pandas as pd

csv = os.path.join(input_folder, 'nucleus_cytoplasm_single_ensg.csv')
nucleus_cytoplasm_single_ensg = pd.read_csv(csv)
print(nucleus_cytoplasm_single_ensg.shape)
nucleus_cytoplasm_single_ensg.head()

(2383, 2)


,ensg_id,level_b
0,ENSG00000049769,nucleoplasm
1,ENSG00000156504,nucleoplasm
2,ENSG00000102096,cytosol
3,ENSG00000068394,nucleoplasm
4,ENSG00000133131,nucleoplasm


In [3]:
nucleus_cytoplasm_single_ensg.empty

False

In [4]:
LEVEL = 'level_b'

nucleus_cytoplasm_single_ensg[f'{LEVEL}_bool'] = 1

In [5]:
target_df = nucleus_cytoplasm_single_ensg.pivot(index='ensg_id', columns=LEVEL)
target_df = target_df.fillna(0)
target_df = target_df.astype(int)
target_df.head()

level_b_bool            
level_b              cytosol nucleoplasm
ensg_id                                 
ENSG00000000003            1           0
ENSG00000001167            0           1
ENSG00000001460            0           1
ENSG00000001461            0           1
ENSG00000002746            1           0

### Check that it's correct with the `head` of the initial data

In [6]:
target_df.loc[nucleus_cytoplasm_single_ensg.ensg_id.head()]

level_b_bool            
level_b              cytosol nucleoplasm
ensg_id                                 
ENSG00000049769            0           1
ENSG00000156504            0           1
ENSG00000102096            1           0
ENSG00000068394            0           1
ENSG00000133131            0           1

## Download and filter sequence data

In [7]:
cd /mnt/data

/mnt/data


In [8]:
mkdir -p genome/hg38/ensembl/v92

In [9]:
cd genome/hg38/ensembl/v92/

/mnt/data/genome/hg38/ensembl/v92


Download ENSEMBL fasta files. I don't understand the difference between `cds` and `cdna` so I downloaded them both. `pep` is the protein sequence

In [10]:
# %%bash

# wget ftp://ftp.ensembl.org/pub/release-92/fasta/homo_sapiens/cds/Homo_sapiens.GRCh38.cds.all.fa.gz
# wget ftp://ftp.ensembl.org/pub/release-92/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
# wget ftp://ftp.ensembl.org/pub/release-92/fasta/homo_sapiens/pep/Homo_sapiens.GRCh38.pep.all.fa.gz

In [11]:
ls

Homo_sapiens.GRCh38.cdna.all.fa        Homo_sapiens.GRCh38.cds.all.fa.gz.1
Homo_sapiens.GRCh38.cdna.all.fa.gz     Homo_sapiens.GRCh38.cds.all.fa.test
Homo_sapiens.GRCh38.cdna.all.fa.gz.1   Homo_sapiens.GRCh38.cds.all.fa.train
Homo_sapiens.GRCh38.cdna.all.fa.test   Homo_sapiens.GRCh38.pep.all.fa
Homo_sapiens.GRCh38.cdna.all.fa.train  Homo_sapiens.GRCh38.pep.all.fa.gz
Homo_sapiens.GRCh38.cds.all.fa         Homo_sapiens.GRCh38.pep.all.fa.test
Homo_sapiens.GRCh38.cds.all.fa.gz      Homo_sapiens.GRCh38.pep.all.fa.train


Unzip the files

In [12]:
# ! gunzip --keep *.gz

In [13]:
ls -lha

total 798M
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 16 23:08 ./
drwxrwxr-x 5 ubuntu ubuntu 4.0K May 16 21:43 ../
-rw-rw-r-- 1 ubuntu ubuntu 341M May 16 21:21 Homo_sapiens.GRCh38.cdna.all.fa
-rw-rw-r-- 1 ubuntu ubuntu  64M May 16 21:21 Homo_sapiens.GRCh38.cdna.all.fa.gz
-rw-rw-r-- 1 ubuntu ubuntu  31M May 16 21:32 Homo_sapiens.GRCh38.cdna.all.fa.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  12M May 16 23:08 Homo_sapiens.GRCh38.cdna.all.fa.test
-rw-rw-r-- 1 ubuntu ubuntu  48M May 16 23:08 Homo_sapiens.GRCh38.cdna.all.fa.train
-rw-rw-r-- 1 ubuntu ubuntu 145M May 16 21:21 Homo_sapiens.GRCh38.cds.all.fa
-rw-rw-r-- 1 ubuntu ubuntu  21M May 16 21:21 Homo_sapiens.GRCh38.cds.all.fa.gz
-rw-rw-r-- 1 ubuntu ubuntu  21M May 16 21:31 Homo_sapiens.GRCh38.cds.all.fa.gz.1
-rw-rw-r-- 1 ubuntu ubuntu 5.0M May 16 23:08 Homo_sapiens.GRCh38.cds.all.fa.test
-rw-rw-r-- 1 ubuntu ubuntu  21M May 16 23:08 Homo_sapiens.GRCh38.cds.all.fa.train
-rw-rw-r-- 1 ubuntu ubuntu  69M May 16 21:21 Homo_sapiens.GRCh38.pep.all.fa
-rw-rw-r-- 1 

In [14]:
! head *.fa

==> Homo_sapiens.GRCh38.cdna.all.fa <==
>ENST00000434970.2 cdna chromosome:GRCh38:14:22439007:22439015:1 gene:ENSG00000237235.2 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:TRDD2 description:T cell receptor delta diversity 2 [Source:HGNC Symbol;Acc:HGNC:12255]
CCTTCCTAC
>ENST00000448914.1 cdna chromosome:GRCh38:14:22449113:22449125:1 gene:ENSG00000228985.1 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:TRDD3 description:T cell receptor delta diversity 3 [Source:HGNC Symbol;Acc:HGNC:12256]
ACTGGGGGATACG
>ENST00000415118.1 cdna chromosome:GRCh38:14:22438547:22438554:1 gene:ENSG00000223997.1 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:TRDD1 description:T cell receptor delta diversity 1 [Source:HGNC Symbol;Acc:HGNC:12254]
GAAATAGT
>ENST00000632684.1 cdna chromosome:GRCh38:7:142786213:142786224:1 gene:ENSG00000282431.1 gene_biotype:TR_D_gene transcript_biotype:TR_D_gene gene_symbol:TRBD1 description:T cell receptor beta diversity 1 [So

Good, the fasta descriptione includes the ENSG ID

In [15]:
fastas = ['Homo_sapiens.GRCh38.cdna.all.fa', 'Homo_sapiens.GRCh38.cds.all.fa', 'Homo_sapiens.GRCh38.pep.all.fa']

In [16]:
%%time

from Bio import SeqIO

for fasta in fastas:
    print(fasta)
#     input_fasta = os.path.join('/mnt/data/', fasta)
    output_fasta = os.path.join(data_folder, f"{fasta}.nuclear_or_cytoplasmic")
    ensgs = []

    nuclear_or_cytoplasmic = []
    for record in SeqIO.parse(fasta, "fasta"):
        for ensg_id in nucleus_cytoplasm_single_ensg['ensg_id']:
            if ensg_id in record.description:
                nuclear_or_cytoplasmic.append(record)
                ensgs.append(ensg_id)
    print(f'\t{len(nuclear_or_cytoplasmic)} sequences found for {len(set(ensgs))} ENSEMBL ids') 
    csv = os.path.join(data_folder, f"{fasta}.nuclear_or_cytoplasmic.target.tsv")
    target_df.loc[ensgs].to_csv(csv, index=False, header=False, sep='\t')
    SeqIO.write(nuclear_or_cytoplasmic, output_fasta, "fasta")


Homo_sapiens.GRCh38.cdna.all.fa
	21588 sequences found for 2377 ENSEMBL ids
Homo_sapiens.GRCh38.cds.all.fa
	13494 sequences found for 2376 ENSEMBL ids
Homo_sapiens.GRCh38.pep.all.fa
	13494 sequences found for 2376 ENSEMBL ids
CPU times: user 3min 16s, sys: 2.39 s, total: 3min 18s
Wall time: 3min 19s


In [17]:
target_df.head()

level_b_bool            
level_b              cytosol nucleoplasm
ensg_id                                 
ENSG00000000003            1           0
ENSG00000001167            0           1
ENSG00000001460            0           1
ENSG00000001461            0           1
ENSG00000002746            1           0

In [20]:
ls -lha /mnt/data

total 130M
drwxrwxrwx 13 ubuntu root   4.0K May 18 19:07 ./
drwxrwxrwx  4 root   root   4.0K Apr 24 23:53 ../
drwxr-xr-x  3 root   root   4.0K Apr 25 00:38 data/
drwxrwxrwx  3 ubuntu ubuntu 4.0K Apr 24 23:25 fastq/
drwxrwxr-x  2 ubuntu ubuntu 4.0K May 18 18:55 fastq_dump_v2/
drwxrwxr-x  2 ubuntu ubuntu 4.0K May 18 18:54 fastq_dump_v3/
drwxrwxr-x  3 ubuntu ubuntu 4.0K May 18 19:07 figures/
drwxrwxr-x  4 ubuntu ubuntu 4.0K May 16 21:20 genome/
drwxr-xr-x  2 root   root   4.0K Apr 24 23:53 hca/
-rw-rw-r--  1 ubuntu ubuntu  80M May 16 21:35 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic
-rw-rw-r--  1 ubuntu ubuntu  34M May 16 21:37 Homo_sapiens.GRCh38.cds.all.fa.nuclear_or_cytoplasmic
-rw-rw-r--  1 ubuntu ubuntu  17M May 16 21:38 Homo_sapiens.GRCh38.pep.all.fa.nuclear_or_cytoplasmic
drwxrwxr-x 25 ubuntu ubuntu 4.0K May 16 20:18 rawdata/
drwxrwxr-x 14 ubuntu ubuntu 4.0K May 14 17:17 sourmash/
drwxrwxr-x  3 ubuntu ubuntu 4.0K Apr 25 17:20 trinity-output/
drwxrwxr-x  3 ubuntu ubuntu 4

In [21]:
! echo $data_folder

/home/ubuntu/code/sequence-localization/data/021_get_nucleus_cytoplasm_sequences


In [22]:
! cp /mnt/data/*nuclear_or_cytoplasmic $data_folder

In [23]:
! ls -lha $data_folder

total 226M
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 16 23:10 .
drwxrwxr-x 8 ubuntu ubuntu 4.0K May 16 21:31 ..
-rw-rw-r-- 1 ubuntu ubuntu  80M May 18 19:54 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic
-rw-rw-r-- 1 ubuntu ubuntu 169K May 18 19:53 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.target.tsv
-rw-rw-r-- 1 ubuntu ubuntu  25K May 16 23:09 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.target.tsv.test
-rw-rw-r-- 1 ubuntu ubuntu  97K May 16 23:09 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.target.tsv.train
-rw-rw-r-- 1 ubuntu ubuntu  12M May 16 23:09 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.test
-rw-rw-r-- 1 ubuntu ubuntu  48M May 16 23:09 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.train
-rw-rw-r-- 1 ubuntu ubuntu  34M May 18 19:54 Homo_sapiens.GRCh38.cds.all.fa.nuclear_or_cytoplasmic
-rw-rw-r-- 1 ubuntu ubuntu 106K May 18 19:53 Homo_sapiens.GRCh38.cds.all.fa.nuclear_or_cytoplasmic.target.tsv
-rw-rw-r-- 1 ubuntu ubuntu  15

In [24]:
! head $data_folder/*

==> /home/ubuntu/code/sequence-localization/data/021_get_nucleus_cytoplasm_sequences/Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic <==
>ENST00000419783.2 cdna chromosome:GRCh38:3:49357171:49358600:-1 gene:ENSG00000233276.4 gene_biotype:polymorphic_pseudogene transcript_biotype:protein_coding gene_symbol:GPX1 description:glutathione peroxidase 1 [Source:HGNC Symbol;Acc:HGNC:4553]
GAGCCCTCGAGGGCCCCAGCCCTTGGAAGGGTAACCTGGACCGCTGCCGCCTGGTTGCCT
GGGCCAGACCAGACATGCCTGCTGCTCCTTCCGGCTTAGGAGGAGCACGCGTCCCGCTCG
GGCGCACTCTCCAGCCTTTTCCTGGCTGAGGAGGGGCCGAGCCCTCCGGGTAGGGCGGGG
GCCGGATGAGGCGGGACCCTCAGGCCCGGAAAACTGCCTGTGCCACGTGACCCGCCGCCG
GCCAGTTAAAAGGAGGCGCCTGCTGGCCTCCCCTTACAGTGCTTGTTCGGGGCGCTCCGC
TGGCTTCTTGGACAATTGCGCCATGTGTGCTGCTCGGCTAGCGGCGGCGGCGGCGGCGGC
CCAGTCGGTGTATGCCTTCTCGGCGCGCCCGCTGGCCGGCGGGGAGCCTGTGAGCCTGGG
CTCCCTGCGGGGCAAGGTACTACTTATCGAGAATGTGGCGTCCCTCTGAGGCACCACGGT
CCGGGACTACACCCAGATGAACGAGCTGCAGCGGCGCCTCGGACCCCGGGGCCTGGTGGT

==> /home/ubuntu/code/sequence-localization/data/021_get_nu

### Make train and test sets

In [25]:
data_folder

'/home/ubuntu/code/sequence-localization/data/021_get_nucleus_cytoplasm_sequences'

In [1]:
# %%time

import numpy as np

np.random.seed(0)

# %%time

from Bio import SeqIO

train_fraction = 0.8

for fasta in fastas:
    input_fasta = os.path.join(data_folder, f"{fasta}.nuclear_or_cytoplasmic")
    print(input_fasta)
    csv = os.path.join(data_folder,f"{fasta}.nuclear_or_cytoplasmic.target.tsv")
    target = pd.read_table(csv, header=None)
    print('\ttarget.shape', target.shape)
    
    total = len(target.index)
    train_index = np.random.random_sample(total) < 0.8
    print("\ttrain_index.sum():", train_index.sum())

    train_target = target.loc[train_index]
    test_target = target.loc[~train_index]
    
    valid_index = train_index
    
    print("\ttest_target.shape:", test_target.shape, f'{100*float(len(test_target.index))/total}')
    print("\ttrain_target.shape:", train_target.shape)
    
    test_target.to_csv(csv + ".test", index=False, header=False, sep='\t')
    train_target.to_csv(csv + ".train", index=False, header=False, sep='\t')
    
    test_records = []
    train_records = []
    
#     for i, record in enumerate(SeqIO.parse(input_fasta, "fasta")):
#         if train_index[i]:
#             train_records.append(record)
#         else:
#             test_records.append(record)
    
#     SeqIO.write(test_records, input_fasta + ".test", "fasta")
#     SeqIO.write(train_records, input_fasta + ".train", "fasta")

NameError: name 'fastas' is not defined

In [20]:
target.head()

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


In [21]:
test_target.tail()

,0,1
13458,0,1
13467,1,0
13468,1,0
13484,1,0
13487,1,0


In [22]:
ls -lha $data_folder

total 137M
drwxrwxr-x 2 ubuntu ubuntu 4.0K May 16 23:10 ./
drwxrwxr-x 8 ubuntu ubuntu 4.0K May 16 21:31 ../
-rw-rw-r-- 1 ubuntu ubuntu  42M May 18 21:40 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic
-rw-rw-r-- 1 ubuntu ubuntu  85K May 18 21:40 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.target.tsv
-rw-rw-r-- 1 ubuntu ubuntu  17K May 18 23:20 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.target.tsv.test
-rw-rw-r-- 1 ubuntu ubuntu  68K May 18 23:20 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.target.tsv.train
-rw-rw-r-- 1 ubuntu ubuntu 8.4M May 18 23:20 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.test
-rw-rw-r-- 1 ubuntu ubuntu  34M May 18 23:20 Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic.train
-rw-rw-r-- 1 ubuntu ubuntu  18M May 18 21:41 Homo_sapiens.GRCh38.cds.all.fa.nuclear_or_cytoplasmic
-rw-rw-r-- 1 ubuntu ubuntu  53K May 18 21:41 Homo_sapiens.GRCh38.cds.all.fa.nuclear_or_cytoplasmic.target.tsv
-rw-rw-r-- 1 ubuntu ubuntu  

In [23]:
! head $data_folder/*nuclear_or_cytoplasmic

==> /home/ubuntu/code/sequence-localization/data/021_get_nucleus_cytoplasm_sequences/Homo_sapiens.GRCh38.cdna.all.fa.nuclear_or_cytoplasmic <==
>ENST00000419783.2 cdna chromosome:GRCh38:3:49357171:49358600:-1 gene:ENSG00000233276.4 gene_biotype:polymorphic_pseudogene transcript_biotype:protein_coding gene_symbol:GPX1 description:glutathione peroxidase 1 [Source:HGNC Symbol;Acc:HGNC:4553]
GAGCCCTCGAGGGCCCCAGCCCTTGGAAGGGTAACCTGGACCGCTGCCGCCTGGTTGCCT
GGGCCAGACCAGACATGCCTGCTGCTCCTTCCGGCTTAGGAGGAGCACGCGTCCCGCTCG
GGCGCACTCTCCAGCCTTTTCCTGGCTGAGGAGGGGCCGAGCCCTCCGGGTAGGGCGGGG
GCCGGATGAGGCGGGACCCTCAGGCCCGGAAAACTGCCTGTGCCACGTGACCCGCCGCCG
GCCAGTTAAAAGGAGGCGCCTGCTGGCCTCCCCTTACAGTGCTTGTTCGGGGCGCTCCGC
TGGCTTCTTGGACAATTGCGCCATGTGTGCTGCTCGGCTAGCGGCGGCGGCGGCGGCGGC
CCAGTCGGTGTATGCCTTCTCGGCGCGCCCGCTGGCCGGCGGGGAGCCTGTGAGCCTGGG
CTCCCTGCGGGGCAAGGTACTACTTATCGAGAATGTGGCGTCCCTCTGAGGCACCACGGT
CCGGGACTACACCCAGATGAACGAGCTGCAGCGGCGCCTCGGACCCCGGGGCCTGGTGGT

==> /home/ubuntu/code/sequence-localization/data/021_get_nu

Search for `ATGTGTGCTGCT`, the very beginning of the CDS. It shows that the CDS is the pure coding sequence (exons only) while the CDNA is the whole CDNA molecule, including introns and UTRs. We want the whole UTRs too.